In [61]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pdb
from Patch_ana import patch_pip, spon_ana
import seaborn as sns
import pickle
from matplotlib.backends.backend_pdf import PdfPages
plt.style.use('classic')

% matplotlib inline
# % matplotlib notebook

%load_ext autoreload
% autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
post_2Hz_reci = pd.read_pickle('/data/ana_post_2Hz_reci')

In [14]:
post_2Hz_reci

,Before,After,File name,elimi,spon_elimi_bef,spon_elimi_aft
121,"{'ave_amp1': [[4.80637267564e-11], [1.87817988...","{'ave_amp1': [[4.25482222695e-11], [4.15985512...",2017081802,30,"[0, 1, 15, 20, 21, 22, 25, 32, 39, 42, 78]","[52, 57, 64, 65]"
122,"{'ave_amp1': [[9.19310355367e-11], [6.84142632...","{'ave_amp1': [[7.71111564184e-11], [6.49876114...",2017082302,50,"[0, 3, 24, 25, 26, 37, 47, 50]","[67, 73, 80, 81, 82, 84, 88, 90, 96, 101, 116,..."
123,"{'ave_amp1': [[9.69784397064e-11], [8.21152440...","{'ave_amp1': [[4.01764970795e-11], [4.81833983...",2017082304,40,"[16, 18, 19]","[38, 44, 50, 56, 57, 71, 84, 88, 95]"
124,"{'ave_amp1': [[3.5281256566e-11], [6.337313946...","{'ave_amp1': [[5.70692305462e-11], [5.37819591...",2017082402,40,"[0, 2, 4, 10, 35, 54]",[37]
125,"{'ave_amp1': [[9.15554304815e-11], [1.06799826...","{'ave_amp1': [[5.10411496332e-11], [4.89674547...",2017083001,45,"[0, 9, 20, 22, 25, 31, 35, 49, 52, 56, 59]","[57, 63, 67, 70, 71, 80, 104]"
126,"{'ave_amp1': [[9.03576852971e-12], [1.56598111...","{'ave_amp1': [[1.4636456127e-11], [1.187144359...",2017083101,30,"[3, 5, 12, 13, 14, 15, 16, 19, 24, 44, 50, 67,...","[37, 39, 45, 46, 59, 60]"


In [68]:
# post_2Hz_reci.index
bef_aft_choice = 'after'

# list(post_2Hz_reci.index)

for choice in list(post_2Hz_reci.index):
    
    if bef_aft_choice == 'before':
        spon_list = post_2Hz_reci.Before.loc[choice]['spon_trace'].copy()
        elimi_list = post_2Hz_reci.loc[choice]['spon_elimi_bef'].copy()
        spon_list = spon_ana.pop_Spon_Trace(spon_list, elimi_list)
        spon_trace = -1 * np.hstack(spon_list)

        plt.figure()
        plt.plot(spon_trace)

        # Obtain the template
        popt, xdata, template = spon_ana.template_Gen(post_2Hz_reci, index=choice, bef_aft=0)
    else:
        spon_list = post_2Hz_reci.After.loc[choice]['spon_trace'].copy()
        elimi_list = post_2Hz_reci.loc[choice]['spon_elimi_aft'].copy()
        
        range_idx = spon_ana.post_bef_end(post_2Hz_reci.loc[choice].elimi)
        spon_list = [spon_list[i] for i in range_idx]
        elimi_list = list(np.array(elimi_list) - range_idx[0])
        
        spon_list = spon_ana.pop_Spon_Trace(spon_list, elimi_list)
        spon_trace = -1 * np.hstack(spon_list)

        plt.figure()
        plt.plot(spon_trace)

        # Obtain the template
        popt, xdata, template = spon_ana.template_Gen(post_2Hz_reci, index=choice, bef_aft=1)


    plt.figure()
    plt.plot(template)
    plt.plot(spon_ana.fun_exp(xdata, *popt), label='raw fit')
    plt.plot(spon_ana.fun_exp(xdata, *popt*2), label='scale up')
    plt.plot(spon_ana.fun_exp(xdata, *popt/2), label='scale down')
    plt.legend()
    
    psc_dic = {}
    
    for i in range(len(spon_list)):
        # detection locations of PSCs
        psc_loc_raw = spon_ana.spon_detect(spon_list[i], spon_ana.fun_exp(xdata, *popt))
        psc_loc_up = spon_ana.spon_detect(spon_list[i], spon_ana.fun_exp(xdata, *popt*2))
        psc_loc_down = spon_ana.spon_detect(spon_list[i], spon_ana.fun_exp(xdata, *popt/2))
        psc_list= list(set(np.concatenate([psc_loc_raw,psc_loc_up, psc_loc_down])))
        psc_list.sort()
        psc_loc = np.array(psc_list).reshape(1,-1)
        # Obtain amplitude, decay, onset time constant of each psc
        psc_dic[i] = spon_ana.psc_search(spon_list[i], psc_loc)
    
    # Save the results for future analysis
    with open('/data/psc_test_' + bef_aft_choice + '_{}'.format(choice), 'wb') as fp:
            pickle.dump(psc_dic, fp)
            
    print('saving psc data from %d ...' % choice)
    
    for j in range(len(psc_dic)):
         for i in range(len(psc_dic[j])):
            plt.figure()
            plt.plot(psc_dic[j][i]['trace'])
            plt.title("%d th resp, %d th trace, decay=%s, error=%0.5f  " % (j, i, psc_dic[j][i]['decay_tau'], psc_dic[j][i]['fit_error']*10**23))
            plt.plot(np.arange(len(psc_dic[j][i]['decay_fit']))+np.argmax(psc_dic[j][i]['trace']), psc_dic[j][i]['decay_fit'])

In [50]:
a = 0.02
b = 0.0005 * 10 ** -10

In [51]:
b

5e-14

In [52]:
b > 0.05 * 10 ** -10

False

In [53]:
if a < 0.0003 or a > 0.01 or b > 0.05 * 10 ** -10:
    a = None

In [54]:
print(a)

None



### 1. Create preliminary template through fitting response to stimulation

In [ ]:
with open('/data/psc_test_{}'.format(choice), 'rb') as fp:
    psc_dic = pickle.load(fp)

In [ ]:
amp = []
decay_tau = []
onset_tau = []
for j in range(len(psc_dic)):
    for i in range(len(psc_dic[j])):
        amp.append(psc_dic[j][i]['amp'])
        decay_tau.append(psc_dic[j][i]['decay_tau'])
        onset_tau.append(psc_dic[j][i]['onset_tau'])

In [ ]:
amp_data = np.array(amp)
decay_tau_data = np.array(decay_tau)
onset_tau_data = np.array(onset_tau)

In [ ]:
plt.plot(amp_data, decay_tau, 'o', alpha=0.3)
plt.ylim([0, 0.005])

In [ ]:
plt.hist(decay_tau, range=(0,0.005), bins=100)

In [ ]:
plt.plot(amp_data, onset_tau, 'o')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(np.array(onset_tau), np.array(decay_tau), np.array(amp_data))
plt.show()

### 2. Extract examplary large and isolated events using a high detection threshold

### 3. Create a final template by fitting a function to the average of the exemplary events

### 4. Extract all events with the final template using a low detection criterion threshold

### 5. Eliminate false-positive, add false-negative events

In [ ]:
# Create